In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import env
import wrangle
import explore

import warnings
warnings.filterwarnings("ignore")

from pydataset import data

from sklearn.linear_model import LinearRegression


#### Load the tips dataset

In [7]:
df = data('tips')
target = 'tip'
features = [col for col in df.columns if col != target]

In [9]:
train, test, validate = wrangle.train_test_validate_split(df)

train	 n = 136
test	 n = 49
validate n = 59


In [10]:
x_train = train[features]
y_train = train[target]

x_validate = validate[features]
y_validate = validate[target]

x_test = test[features]
y_test = test[target]

#### Fit a linear regression model (ordinary least squares) and compute yhat, predictions of tip using total_bill.

In [ ]:
# assuming X and y are already defined
model = LinearRegression().fit(x_train, y_train)
train_predictions = model.predict(x_train)